In [ ]:
# from https://www.kaggle.com/code/mrankitgupta/spotify-data-analysis-using-python
# and from # https://github.com/enjuichang/PracticalDataScience-ENCA
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import json
import pandas as pd
import requests
import os


# setting up the credentials, two different methods will be used
client_id = "2ca89022ac6348b99eb6e5cb99583158"
client_secret = "4dd5feb53234451f9952db292972a656"
auth_url = 'https://accounts.spotify.com/api/token'

os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:1410/'

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'user-top-read'
}

# get access token
auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')

# access api
base_url = 'https://api.spotify.com/v1/'

headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}


# store JSON
playlists = response.json().get('playlists').get('items')
recent_plays = response_recent.json()
liked_plays = response_liked.json()

#get playlist ids
playlist_ids = []
for pl in playlists:
    playlist_id = pl.get('id')
    playlist_ids.append(playlist_id)

# get track ids
#recent_ids = []
#for tr in recent_plays:
#    track_id = tr.get('id')
#    recent_ids.append(track_id)
playlists


In [41]:
# Request top tracks data and create top tracks dataframe

# set up authorisation for user data
scope = "user-top-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

#https://www.linkedin.com/pulse/analyse-your-spotify-data-python-spotipy-shai-nisan-ph-d-/
user_top_tracks = sp.current_user_top_tracks(limit=50)
user_top_tracks2 = sp.current_user_top_tracks(limit=50, offset = 50)

# create lists for future iteration
top_track_ids = []
top_track_titles = []

for i in user_top_tracks['items']:
    top_track_ids.append(i['id'])
    top_track_titles.append(i['name'])
for i in user_top_tracks2['items']:
    top_track_ids.append(i['id'])
    top_track_titles.append(i['name'])

#create a dataframe
top_features = sp.audio_features(top_track_ids)
top_features_df = pd.DataFrame(data=top_features, columns=top_features[0].keys())
top_features_df.drop(columns=["type", "id","uri","track_href", "analysis_url", "time_signature"], inplace=True)

top_features_df



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.399,0.7610,9,-6.318,1,0.0334,0.000105,0.045600,0.0757,0.2430,140.084,304907
1,0.586,0.5660,9,-16.305,1,0.0398,0.011000,0.000124,0.0741,0.5850,99.980,289027
2,0.252,0.1880,1,-11.648,1,0.0456,0.329000,0.121000,0.1000,0.0302,86.997,113773
3,0.408,0.7600,6,-6.844,1,0.0316,0.000973,0.000073,0.1120,0.0380,105.008,322760
4,0.404,0.4070,5,-11.843,1,0.0299,0.492000,0.033800,0.3910,0.0848,106.547,189293
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.748,0.2680,11,-9.687,1,0.0299,0.839000,0.002650,0.1260,0.1930,118.017,257160
96,0.114,0.1980,3,-14.357,1,0.0313,0.818000,0.917000,0.1080,0.0378,89.535,393427
97,0.539,0.8840,0,-5.311,1,0.0435,0.260000,0.575000,0.3040,0.1680,120.035,236360
98,0.471,0.5340,7,-7.900,1,0.0402,0.004710,0.004490,0.1650,0.5290,183.962,207507


In [ ]:
# Request featured (recommended) playlists

# getting the featured playlists
featured_playlists_endpoint = 'browse/featured-playlists/?limit=50'
featured_playlists_url = ''.join([base_url,featured_playlists_endpoint])

# send API request
featured_response = requests.get(featured_playlists_url,headers=headers)

# store JSON
featured_playlists = featured_response.json()

featured_playlists


In [ ]:

top_tracks = {"track":[],"album":[],"artist":[],"ID":[],"popularity":[],"release_date":[],"duration_ms":[], "artist_id":[]}

for i in user_top_tracks["items"]:
    
    top_tracks["track"].append(i['name'])
    top_tracks["album"].append(i['album']['name'])
    top_tracks["artist"].append(i['artists'][0]['name'])
    top_tracks["ID"].append(i['id'])
    top_tracks["popularity"].append(i['popularity'])
    top_tracks["release_date"].append(i['album']['release_date'])
    top_tracks["duration_ms"].append(i['duration_ms'])
    top_tracks["artist_id"].append(i['artists'][0]['id']) 

for i in user_top_tracks2["items"]:
    
    top_tracks["track"].append(i['name'])
    top_tracks["album"].append(i['album']['name'])
    top_tracks["artist"].append(i['artists'][0]['name'])
    top_tracks["ID"].append(i['id'])
    top_tracks["popularity"].append(i['popularity'])
    top_tracks["release_date"].append(i['album']['release_date'])
    top_tracks["duration_ms"].append(i['duration_ms'])
    top_tracks["artist_id"].append(i['artists'][0]['id']) 


user_top_tracks

top_tracks

df = pd.DataFrame.from_dict(top_tracks) 

df

In [21]:
#sp.audio_features(top_tracks["ID"])
#features
user_top_tracks['items'][0]

{'album': {'album_type': 'ALBUM',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/63MQldklfxkjYDoUE4Tppz'},
    'href': 'https://api.spotify.com/v1/artists/63MQldklfxkjYDoUE4Tppz',
    'id': '63MQldklfxkjYDoUE4Tppz',
    'name': 'M83',
    'type': 'artist',
    'uri': 'spotify:artist:63MQldklfxkjYDoUE4Tppz'}],
  'available_markets': ['CA', 'PR', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/4HshNHpHWtmaw8sDrOXNuE'},
  'href': 'https://api.spotify.com/v1/albums/4HshNHpHWtmaw8sDrOXNuE',
  'id': '4HshNHpHWtmaw8sDrOXNuE',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2739b4275c2e7153d19d45884fa',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e029b4275c2e7153d19d45884fa',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048519b4275c2e7153d19d45884fa',
    'width': 64}],
  'name': 'Saturdays = Youth',
  'release_date': '2008-04-15',


In [ ]:
endpoint = "me/top/tracks"
url = ''.join([base_url,endpoint])
response = requests.get(url,headers=headers)
response.json()


In [30]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_id = playlist_ids[0]
playlist_tracks_data = sp.playlist_tracks(playlist_id)

playlist_tracks_id = []
playlist_tracks_titles = []
playlist_tracks_artists = []
playlist_tracks_first_artists = []

    #go through the dictionary to extract the data
for track in playlist_tracks_data['items']:
    playlist_tracks_id.append(track['track']['id'])
    playlist_tracks_titles.append(track['track']['name'])
    # adds a list of all artists involved in the song to the list of artists for the playlist
    artist_list = []
    for artist in track['track']['artists']:
        artist_list.append(artist['name'])
    playlist_tracks_artists.append(artist_list)
    playlist_tracks_first_artists.append(artist_list[0])

    #create a dataframe
features = sp.audio_features(playlist_tracks_id)
features_df = pd.DataFrame(data=features, columns=features[0].keys())
features_df['title'] = playlist_tracks_titles
features_df['first_artist'] = playlist_tracks_first_artists
features_df['all_artists'] = playlist_tracks_artists
features_df = features_df[['id', 'title', 'first_artist', 'all_artists',
                            'danceability', 'energy', 'key', 'loudness',
                            'mode', 'acousticness', 'instrumentalness',
                            'liveness', 'valence', 'tempo',
                            'duration_ms', 'time_signature']]

features_df.head()

,id,title,first_artist,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0h3Xy4V4apMraB5NuM8U7Z,Stumblin' In,CYRIL,[CYRIL],0.875,0.518,2,-8.388,1,0.0638,0.009950,0.125,0.370,126.089,213364,4
1,6tNQ70jh4OwmPGpYy6R2o9,Beautiful Things,Benson Boone,[Benson Boone],0.472,0.471,10,-5.692,1,0.1510,0.000000,0.140,0.219,105.029,180304,3
2,1UP7PXne1lIYZB5G5aiRGL,II MOST WANTED,Beyoncé,"[Beyoncé, Miley Cyrus]",0.533,0.355,8,-6.132,1,0.4970,0.000000,0.174,0.256,137.986,208521,4
3,17phhZDn6oGtzMe56NuWvj,Lose Control,Teddy Swims,[Teddy Swims],0.561,0.604,9,-4.409,1,0.1990,0.000019,0.104,0.242,159.920,210689,3
4,6tNgRQ0K2NYZ0Rb9l9DzL8,obsessed,Olivia Rodrigo,[Olivia Rodrigo],0.683,0.587,8,-5.094,1,0.0219,0.000000,0.305,0.697,119.985,170542,4
